In [1]:
import nrrd
import numpy as np
import os
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from skimage.segmentation import mark_boundaries
from scipy import ndimage as ndi
from helper import *
import graph_tool.all as gt
import plotly.graph_objects as go
import time

current_directory = os.getcwd()
filename = "manual_2"
label_path = f"{current_directory}/data/label/{filename}_label.nrrd"
raw_data_path = f"{current_directory}/data/raw/{filename}_raw.nrrd"

mask_data, mask_header = nrrd.read(label_path)
raw_data, raw_header = nrrd.read(raw_data_path)

In [2]:
#rotate the data and calculate reduced data representations
#lists of the 3 rotations of the reduced mask and raw data
reduced_mask_data_list = []
reduced_raw_data_list = []

reduced_mask_data_list.append(coarsen_image(mask_data, 3))
reduced_raw_data_list.append(coarsen_image(raw_data, 3))

k = 1 # number of times to rotate the array
axis = 0 #0 for x, 1 for y and 2 for z
mask_data_r = np.rot90(mask_data, k, axes=(axis, (axis+1)%3))
raw_data_r = np.rot90(raw_data, k, axes=(axis, (axis+1)%3))

reduced_mask_data_list.append(coarsen_image(mask_data_r, 3))
reduced_raw_data_list.append(coarsen_image(raw_data_r, 3))

axis = 1
mask_data_r = np.rot90(mask_data, k, axes=(axis, (axis+1)%3))
raw_data_r = np.rot90(raw_data, k, axes=(axis, (axis+1)%3))

reduced_mask_data_list.append(coarsen_image(mask_data_r, 3))
reduced_raw_data_list.append(coarsen_image(raw_data_r, 3))
num_rotations = 3
#index 0 of a list element is full res, each further index is 2x lower res, 256, 128, 64, 32

In [3]:
#visualise data to confirm alignment
res_i = 0
rot_i = 2
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(reduced_raw_data_list[rot_i][res_i][:,slice_index,:], reduced_mask_data_list[rot_i][res_i][:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(reduced_raw_data_list[rot_i][res_i][:,:,slice_index], reduced_mask_data_list[rot_i][res_i][:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(reduced_raw_data_list[rot_i][res_i][slice_index,:,:], reduced_mask_data_list[rot_i][res_i][slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=reduced_raw_data_list[rot_i][0].shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [4]:
# Helper functions that are annoying to move to a seperate file
def calculate_seam_iter(directed_graph, src, tgt, weights, test_size, x_pos, y_pos, z_pos):
    # Compute the residual capactiy of the edges
    res = gt.boykov_kolmogorov_max_flow(directed_graph, src, tgt, weights) #time complexity: edges * vertices^2 * abs(min cut) 
    #use the residual graph to get the max flow
    flow = sum(weights[e] - res[e] for e in tgt.in_edges())
    # Determine the minimum cut partition
    part = gt.min_st_cut(directed_graph, src, weights, res)

    # Find the boundary vertices
    boundary_vertices = find_boundary_vertices(np.array(directed_graph.get_edges()), part)
    shape = (test_size, test_size, test_size)

    # Convert the boundary vertices to a 3D array
    boundary_array = boundary_vertices_to_array_masked(boundary_vertices, shape, 'x', x_pos, y_pos, z_pos)
    
    return boundary_array, flow

def multi_res_seam_iter(res_index, mask_array_data, b_arr_up):
    masked_array = mask_array_data[res_index].copy().astype(np.int16)
    masked_array[b_arr_up == 0] = -1
    stime= time.time()
    directed_graph, src, tgt, weights, x_pos, y_pos, z_pos = create_masked_directed_energy_graph_from_mask(masked_array)
    # print(f"Time taken to create graph {res_index}:", time.time()-stime)
    boundary_array, flow = calculate_seam_iter(directed_graph, src, tgt, weights, masked_array.shape[0], x_pos, y_pos, z_pos)
    return boundary_array

def multi_res_seam_calculation(mask_array_data, res_index=3, upscale_factor=2, dilation_amount=1):
    directed_graph, src, tgt, weights, x_pos, y_pos, z_pos = create_masked_directed_energy_graph_from_mask(mask_array_data[res_index])
    boundary_array, flow = calculate_seam_iter(directed_graph, src, tgt, weights, mask_array_data[res_index].shape[0], x_pos, y_pos, z_pos)
    b_arr_up = upscale_and_dilate_3d(boundary_array, upscale_factor=2, dilation_amount=dilation_amount)
    for i in range(res_index-1, -1, -1):
        boundary_array = multi_res_seam_iter(i, mask_array_data, b_arr_up)
        if i != 0:
            b_arr_up = upscale_and_dilate_3d(boundary_array, upscale_factor=upscale_factor, dilation_amount=dilation_amount)
    return boundary_array

In [5]:
def save_nrrd(mask_array_data, raw_array_data, filename, num_seams_removed, rot):
    output_dir = os.path.join(os.getcwd(), 'output/densified_cubes')
    os.makedirs(output_dir, exist_ok=True)
    # Save mask_array_data[0] as NRRD with a timestamp
    mask_nrrd_path = os.path.join(output_dir, f'{filename}_{num_seams_removed}_{rot}_densified_label.nrrd')
    nrrd.write(mask_nrrd_path, mask_array_data[0])
    print(f"Saved mask_array_data[0] to {mask_nrrd_path}")

    # Save raw_array_data as NRRD with a timestamp
    raw_nrrd_path = os.path.join(output_dir, f'{filename}_{num_seams_removed}_{rot}_densified_data.nrrd')
    nrrd.write(raw_nrrd_path, raw_array_data)
    print(f"Saved raw_array_data to {raw_nrrd_path}")

In [8]:
# Define the function to be executed in parallel
def process_data(k, reduced_mask_data_list, reduced_raw_data_list, filename, num_seams_to_remove=120, res_index=3, save_interval=20):
    print(f"Processing data for k={k}")
    mask_array_data = reduced_mask_data_list[k]
    raw_array_data = reduced_raw_data_list[k][0]
    boundary_arrays = []

    for i in range(num_seams_to_remove):
        stime = time.time()
        boundary_array = multi_res_seam_calculation(mask_array_data, res_index, dilation_amount=1)
        mask_array_data, raw_array_data = remove_voxels(mask_array_data[0], raw_array_data, boundary_array)
        mask_array_data = coarsen_image(mask_array_data, res_index)
        boundary_arrays.append(boundary_array)
        print(f"Time taken to calculate and remove seam {i} for k={k}:", time.time() - stime)
        if i !=0 and (i % save_interval == 0 or i == 76):
            # timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            # filename = f"output_{k}_{i}_{timestamp}.nrrd"
            save_nrrd(mask_array_data, raw_array_data, filename, i, k % num_rotations)

In [10]:
# Parallel execution using ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
def main():
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_data, k, reduced_mask_data_list, reduced_raw_data_list) for k in range(len(reduced_mask_data_list))]

    # Wait for all futures to complete
    for future in futures:
        future.result()

if __name__ == '__main__':
    main()

Process SpawnProcess-4:
Traceback (most recent call last):
  File "/Users/jamesdarby/anaconda3/envs/3d_sheet_carving/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/jamesdarby/anaconda3/envs/3d_sheet_carving/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jamesdarby/anaconda3/envs/3d_sheet_carving/lib/python3.12/concurrent/futures/process.py", line 251, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jamesdarby/anaconda3/envs/3d_sheet_carving/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'process_data' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [11]:
for k in range(len(reduced_mask_data_list)):
    res_index = 3 #res index to start at, higher values are lower res, and thus should be faster
    mask_array_data = reduced_mask_data_list[i]
    raw_array_data = reduced_raw_data_list[i][0]
    num_seams_to_remove = 120 #76 to hit the 180 vram limit -> mixed precision could increase this
    boundary_arrays = []
    save_interval = 20

    for i in range(num_seams_to_remove):
        stime = time.time()
        boundary_array = multi_res_seam_calculation(mask_array_data, res_index, dilation_amount=1)
        mask_array_data, raw_array_data = remove_voxels(mask_array_data[0], raw_array_data, boundary_array)
        mask_array_data = coarsen_image(mask_array_data, res_index)
        boundary_arrays.append(boundary_array)
        print(f"Time taken to calculate and remove seam {i}:", time.time()-stime)
        if i % save_interval == 0 or i == 76:
            save_nrrd(mask_array_data, raw_array_data, filename, i, k%num_rotations)

Time taken to calculate and remove seam 0: 20.845386028289795
Time taken to calculate and remove seam 1: 20.649846076965332
Time taken to calculate and remove seam 2: 20.21210479736328
Time taken to calculate and remove seam 3: 19.936538219451904
Time taken to calculate and remove seam 4: 20.254873991012573
Time taken to calculate and remove seam 5: 20.076255083084106
Time taken to calculate and remove seam 6: 20.236415147781372
Time taken to calculate and remove seam 7: 20.147073984146118
Time taken to calculate and remove seam 8: 20.458027124404907
Time taken to calculate and remove seam 9: 20.3747079372406
Time taken to calculate and remove seam 10: 19.99093222618103
Time taken to calculate and remove seam 11: 19.889456033706665
Time taken to calculate and remove seam 12: 20.220655918121338
Time taken to calculate and remove seam 13: 20.198872804641724
Time taken to calculate and remove seam 14: 19.854634046554565
Time taken to calculate and remove seam 15: 19.98770570755005
Time ta

In [ ]:
#save results
# Create output directory if it doesn't exist
output_dir = os.path.join(os.getcwd(), 'output/densified_cubes')
os.makedirs(output_dir, exist_ok=True)

# Save mask_array_data[0] as NRRD with a timestamp
mask_nrrd_path = os.path.join(output_dir, f'{filename}_{num_seams_to_remove}_densified_label.nrrd')
nrrd.write(mask_nrrd_path, mask_array_data[0])
print(f"Saved mask_array_data[0] to {mask_nrrd_path}")

# Save raw_array_data as NRRD with a timestamp
raw_nrrd_path = os.path.join(output_dir, f'{filename}_{num_seams_to_remove}_densified_data.nrrd')
nrrd.write(raw_nrrd_path, raw_array_data)
print(f"Saved raw_array_data to {raw_nrrd_path}")

Saved mask_array_data[0] to /Users/jamesdarby/Documents/VesuviusScroll/GP/3D_sheet_carving/output/densified_cubes/densified_label_20240606_214137.nrrd
Saved raw_array_data to /Users/jamesdarby/Documents/VesuviusScroll/GP/3D_sheet_carving/output/densified_cubes/densified_data_20240606_214137.nrrd


In [ ]:
#plot results
res_i = 0
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(raw_array_data[:,slice_index,:], mask_array_data[res_i][:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(raw_array_data[:,:,slice_index], mask_array_data[res_i][:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(raw_array_data[slice_index,:,:], mask_array_data[res_i][slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=raw_array_data.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>